In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize
from scipy.optimize import minimize
# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [159]:
import numpy as np

# La entrada es de 28x28 elementos para las imágenes
input_layer_size = 28 * 28

# 10 etiquetas, de 0 a 9
num_labels = 10

# Cargar datos desde el archivo CSV, ignorando la primera fila que contiene los encabezados
data = np.loadtxt('/content/drive/MyDrive/laboratorios_IA/lab_3/maleModifiers.csv', delimiter=',', skiprows=1)

# La primera columna contiene las etiquetas
y = data[:, 0]

# Las restantes columnas son los pixeles de las imágenes
X = data[:, 1:]

# Normalizar los valores de los píxeles para que estén en el rango [0, 1]
X = X / 255.0

# Establecer el dígito 10 en 0, ya que los datos de MATLAB asignan 10 para el dígito 0
y[y == 11] = 0

m = y.size


In [160]:
print(X[0,:])
print(y)

[0.01176471 0.00784314 0.00784314 0.00784314 0.00784314 0.00784314
 0.00392157 0.00392157 0.00392157 0.00392157 0.00784314 0.00392157
 0.00392157 0.00392157 0.         0.00392157 0.00392157 0.00392157
 0.00392157 0.00392157 0.00784314 0.00392157 0.00784314 0.01568627
 0.         0.         0.         0.00784314 0.00784314 0.00784314
 0.         0.         0.         0.         0.         0.01568627
 0.00784314 0.00784314 0.00392157 0.         0.00392157 0.00784314
 0.00392157 0.00392157 0.00784314 0.00392157 0.00784314 0.00392157
 0.00392157 0.01568627 0.         0.         0.1254902  0.48235294
 0.42352941 0.01176471 0.00392157 0.         0.04705882 0.07058824
 0.0745098  0.07843137 0.03529412 0.         0.00392157 0.01568627
 0.00784314 0.00392157 0.00784314 0.01176471 0.01176471 0.
 0.         0.00784314 0.00784314 0.01568627 0.01176471 0.
 0.09411765 0.57254902 0.92941176 1.         1.         0.01176471
 0.01176471 0.22745098 0.80392157 0.89803922 0.89019608 0.90196078
 0.78823529

In [161]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [162]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

In [4]:

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [5]:
def lrCostFunction(theta, X, y, lambda_):
    m = len(y)
    h = sigmoid(X @ theta)
    J = -1*(1/m)*(np.log(h).T @ y + np.log(1-h).T @ (1-y)) + (lambda_/(2*m)) * np.sum(np.square(theta[1:]))
    return J

In [6]:
# Gradiente de la función de costo para la regresión logística
def lrGradient(theta, X, y, lambda_):
    m = len(y)
    h = sigmoid(X @ theta)
    grad = (1/m) * X.T @ (h - y)
    grad[1:] += (lambda_ / m) * theta[1:]
    return grad

In [15]:
# Función para entrenar el modelo de regresión logística
def trainLogisticRegression(X, y, num_labels, lambda_):
    m, n = X.shape
    all_theta = np.zeros((num_labels, n))

    for c in range(num_labels):
        initial_theta = np.zeros(n)
        result = minimize(lrCostFunction, initial_theta, args=(X, (y == c).astype(int), lambda_), method='CG', jac=lrGradient)
        all_theta[c] = result.x

    return all_theta

In [16]:
# Función para predecir las etiquetas utilizando el modelo entrenado
def predict(X, all_theta):
    probabilities = sigmoid(X @ all_theta.T)
    return np.argmax(probabilities, axis=1)

In [ ]:
# Cargar el conjunto de datos
data = np.loadtxt('/content/drive/MyDrive/laboratorios_IA/lab_3/maleModifiers.csv', delimiter=',', skiprows=1)

# Separar las características (X) de las etiquetas (y)
y = data[:, 0]
X = data[:, 1:]

# Normalizar las características
X = X / 255.0

# Dividir el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo de regresión logística
lambda_ = 0.1  # Parámetro de regularización
num_labels = 11  # Número de clases
all_theta = trainLogisticRegression(X_train, y_train, num_labels, lambda_)

# Predecir las etiquetas para el conjunto de entrenamiento y calcular la precisión
predictions_train = predict(X_train, all_theta)
accuracy_train = np.mean(predictions_train == y_train) * 100
print('Precisión en el conjunto de entrenamiento: {:.2f}%'.format(accuracy_train))

# Predecir las etiquetas para el conjunto de prueba y calcular la precisión
predictions_test = predict(X_test, all_theta)
accuracy_test = np.mean(predictions_test == y_test) * 100
print('Precisión en el conjunto de prueba: {:.2f}%'.format(accuracy_test))